In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# checando se a gpu está disponível senão roda na cpu

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device = torch.device('cpu')

### Prática
1. Aquisição e pré-processamento dos dados
2. Treinamento
* Implementar arquitetura
* Definir otimizadores, métricas e regularizadores
3. Teste (avaliação de desempenho)

# Classificação

#### 1. Aquisição dos dados

In [4]:
transforms = transforms.Compose([
                                 transforms.ToTensor()
])
 
# Loading Data and splitting it into train and validation data
train = datasets.MNIST('', train = True, transform = transforms, download = True)
train, valid = random_split(train,[50000,10000])
 
# Create Dataloader of the above tensor with batch size = 32
trainloader = DataLoader(train, batch_size=32)
validloader = DataLoader(valid, batch_size=32)

testset = datasets.MNIST('', train=False,
                                       download=True, transform=transforms)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 1 , 3)
        self.fc1 = nn.Linear(676, 32)
        self.fc2 = nn.Linear(32, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.softmax(x)
        return x


net = Net()

In [7]:
import torch.optim as optim
import numpy as np

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
min_valid_loss = np.inf

In [10]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    train_loss = 0.0
    for data, labels in trainloader:
        # Transfer Data to GPU if available
        # if torch.cuda.is_available():
        #     data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = net(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate gradients 
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()
     
    valid_loss = 0.0
    net.eval()     # Optional when not using Model Specific layer
    for data, labels in validloader:
        # Transfer Data to GPU if available
        # if torch.cuda.is_available():
        #     data, labels = data.cuda(), labels.cuda()
         
        # Forward Pass
        target = net(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
    training_loss_print = train_loss / len(trainloader)
    valid_loss_print = valid_loss / len(validloader)
    print(f'Epoch {epoch+1} \t\t Training Loss: {training_loss_print} \t\t Validation Loss: {valid_loss_print}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss}--->{valid_loss}) \t Saving The Model')
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(net.state_dict(), 'saved_model.pth')

print('Finished Training')

Epoch 1 		 Training Loss: 0.218672568793632 		 Validation Loss: 0.2095297048517024
Validation Loss Decreased(inf--->65.58279761858284) 	 Saving The Model
Epoch 2 		 Training Loss: 0.17647654295589724 		 Validation Loss: 0.1726793692396662
Validation Loss Decreased(65.58279761858284--->54.048642572015524) 	 Saving The Model
Epoch 3 		 Training Loss: 0.15009658217372912 		 Validation Loss: 0.15906454823887387
Validation Loss Decreased(54.048642572015524--->49.78720359876752) 	 Saving The Model
Epoch 4 		 Training Loss: 0.13246884585524446 		 Validation Loss: 0.14502416244342017
Validation Loss Decreased(49.78720359876752--->45.39256284479052) 	 Saving The Model
Epoch 5 		 Training Loss: 0.11959675874802989 		 Validation Loss: 0.1380088684303811
Validation Loss Decreased(45.39256284479052--->43.196775818709284) 	 Saving The Model
Epoch 6 		 Training Loss: 0.10929642036296212 		 Validation Loss: 0.13557842635615067
Validation Loss Decreased(43.196775818709284--->42.436047449475154) 	 Savin

In [11]:
dataiter = iter(testloader)
images, labels = next(dataiter)

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [13]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)

accuracy_score(labels, predicted)

0.96875